In [30]:
import os
import pandas as pd
import win32com.client
from datetime import datetime
from unidecode import unidecode

In [31]:
try:
    local_folder = os.path.dirname(os.path.abspath(__file__))
except:
    local_folder = os.getcwd()

In [32]:
lista_emails = pd.read_excel('lista_emails.xlsx')

In [33]:
lista_emails.columns = [unidecode(col) for col in lista_emails.columns]

In [34]:
lista_emails['MES_REF'] = lista_emails['MES_REF'].map('{:0>2d}'.format)

lista_emails['REF_FATURA'] = lista_emails['MES_REF'] + '/' + lista_emails['ANO_REF'].map(str)

lista_emails['DATA_VENCIMENTO'] = lista_emails['DATA_VENCIMENTO'].dt.strftime('%d/%m/%Y')

lista_emails['VALOR'] = lista_emails['VALOR'].map('{:,.2f}'.format).replace('\.', '-', regex=True).replace(',', '.', regex=True).replace('-', ',', regex=True)

lista_emails['SUJEITO_A_CORTE_A_PARTIR_DE'] = lista_emails['SUJEITO_A_CORTE_A_PARTIR_DE'].dt.strftime('%d/%m/%Y')

lista_emails['VENCIMENTO_REAVISO'] = lista_emails['VENCIMENTO_REAVISO'].dt.strftime('%d/%m/%Y')

In [35]:
data_hoje = datetime.now().strftime('%d/%m/%Y')

In [36]:
outlook = win32com.client.Dispatch("Outlook.Application")

In [37]:
up_local_folder = os.path.abspath(FR'{local_folder}\..')

In [38]:
for row in lista_emails.itertuples():
    with open(FR'{up_local_folder}\assets\reaviso.html', 'r', encoding='utf8') as f:

        msg = outlook.CreateItem(0)

        msg.Subject = 'REAVISO - ENERGISA MS'

        msg.To = row.EMAIL
        msg.Cc = ''

        layout_email = f.read()

        layout_email = layout_email.replace('<var>NOME_CLIENTE</var>', row.NOME_CLIENTE)
        layout_email = layout_email.replace('<var>DATA_HOJE</var>', data_hoje)
        layout_email = layout_email.replace('<var>NUMCDC</var>', str(row.UC))
        layout_email = layout_email.replace('<var>VCTO_REAVISO</var>', row.VENCIMENTO_REAVISO)
        layout_email = layout_email.replace('<var>REF_FAT</var>', row.REF_FATURA)
        layout_email = layout_email.replace('<var>VCTO_FAT</var>', row.DATA_VENCIMENTO)
        layout_email = layout_email.replace('<var>MES_FAT</var>', str(row.MES_REF))
        layout_email = layout_email.replace('<var>VLR_EMIS</var>', row.VALOR)
        layout_email = layout_email.replace('<var>DATA_CORTE</var>', row.SUJEITO_A_CORTE_A_PARTIR_DE)
        layout_email = layout_email.replace('<var>ENDERECO_CLIENTE</var>', row.ENDERECO)
        layout_email = layout_email.replace('_CAMINHO_IMAGENS_', up_local_folder)

        msg.HTMLBody = layout_email

        msg.Display()
        msg.Send()

In [8]:
# preciso adicionar a NOMECSD na df1 só que paraa cruzar as informações preciso que use duas chaves 'NUMCDC_CSD' , 'UC_CS' da df e a 'NUM_CDC_VINCULADA' da df1

import pandas as pd

df = pd.read_feather("D:\Caio - DL\Bases\BASE_CLIENTE.feather")

df.info()

# NUMCDC_CSD UC_CS NOMECSD                      

df1 = pd.read_excel("Entrantes PECLD 1.xlsx")

df1.info()


import pandas as pd

# Supondo que já tenha carregado os DataFrames df e df1 conforme seu código

# Renomeando a coluna em df1 para combinar com a coluna em df
df1.rename(columns={'NUM_CDC_VINCULADA': 'NUMCDC_CSD'}, inplace=True)

# Realizando o merge (cruzamento) com base nas colunas combinadas
# Aqui estou assumindo que você quer manter todas as linhas de df1 e apenas adicionar informações de df onde possível
df_final = pd.merge(df1, df[['NUMCDC_CSD', 'UC_CS', 'NOMECSD']], on=['NUMCDC_CSD'], how='left')

# Agora df_final contém todas as linhas de df1 com a coluna NOMECSD adicionada de df onde as chaves combinam

# NUM_CDC_VINCULADA

df_final.to_excel("Entrantes PECLD 1.xlsx")
df_final.info()
                   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3152854 entries, 0 to 3152853
Data columns (total 74 columns):
 #   Column                       Dtype         
---  ------                       -----         
 0   NUMCDC_CSD                   int64         
 1   REGIONAL                     category      
 2   POLO                         category      
 3   LOCALIDADE                   category      
 4   COD_LOCAL                    int64         
 5   UC_CS                        float64       
 6   SITU                         category      
 7   CODCLS_CLI                   int64         
 8   GT                           category      
 9   CODATIV                      int64         
 10  DESCATIV                     category      
 11  SUBCLASSE                    int64         
 12  DSCSUBCLASSE                 object        
 13  DATALIG                      datetime64[ns]
 14  DATADLG                      datetime64[ns]
 15  DATARLG                      datetime64[ns]
 16  